Detect motion with PIL based on the recipe for OpenCV in the reference. This is because installing OpenCV on Raspberry Pi is a bear.

In [26]:
from picamera import PiCamera, array
import numpy as np
from io import BytesIO
from PIL import Image, ImageFilter, ImageMorph, ImageEnhance
import time
import os

Detect motion by imaging in two takes, one after the other, and comparing pixels. Let's write a function to take a snap.

In [15]:
def take_motion_snap(width, height):
    with PiCamera() as Eye:
        time.sleep(1)
        Eye.resolution = (width, height)
        Eye.rotation = 180
        with array.PiRGBArray(camera=Eye) as Stream:
            Eye.exposure_mode = 'auto'
            Eye.awb_mode = 'auto'
            Eye.capture(Stream, format='rgb')
            return Stream.array

In [21]:
test = take_motion_snap(300, 300)
print("Got an image with width, height as {}, {}.".format(test.shape[0], test.shape[1]))
snap = Image.fromarray(test)
snap.show()

Got an image with width, height as 300, 300.


Now we have an RGB image of the specified height and width as a 3-dimensional numpy array. We want to take the difference between two snaps. Write a wrapper function to use ```take_motion_snap(w, h)```, take two snaps and return the computed difference.

In [38]:
def take_two_motion(intervalsec):
    im_one = take_motion_snap(300, 300)
    tic = time.time()
    toc = tic
    while (toc - tic) < intervalsec:
        toc = time.time()
    im_two = take_motion_snap(300, 300)
    im_diff = np.subtract(im_two, im_one)
    return im_diff


In [44]:
test_diff = take_two_motion(6)
print("Got a difference of width, height as {}, {}.".format(test_diff.shape[0], test_diff.shape[1]))
print("The median, mean diff are {:.2f}, {:.2f}.".format(np.median(test_diff), np.mean(test_diff)))
snap = Image.fromarray(test_diff)
snap.show()

/usr/lib/python3/dist-packages/picamera/encoders.py:544: PiCameraResolutionRounded: frame size rounded up from 300x300 to 320x304
  width, height, fwidth, fheight)))


Got a difference of width, height as 300, 300.
The median, mean diff are 3.00, 113.51.


Apply thresholding to remove noise from motion and nuisance effects such as lighting change. In thresholding, we will set the value of a pixel in each of the RGB channels to 0 (min) or 255 (max) accordng to a binary threshold value. In OpenCV, this operation would be ```cv2.threshold(frame_delta, 50, 255, cv2.THRESH_BINARY)```. Use numpy operations here.

In [32]:
def threshold_difference(imdiff, threshold=50):
    return np.uint8(np.where(imdiff > threshold, 255, 0))

In [46]:
diff_clean = threshold_difference(test_diff, np.mean(test_diff))
print("Got a thresholded image of width, height as {}, {}.".format(diff_clean.shape[0], diff_clean.shape[1]))
diff_clean.dtype
snap = Image.fromarray(diff_clean)
snap.show()

Got a thresholded image of width, height as 300, 300.


In [59]:
snap_eroded = snap.filter(ImageFilter.MinFilter(7))
snap_eroded.show()
print("After erosion, got median, mean as {:.2f}, {:.2f}.".format(np.median(snap_eroded), np.mean(snap_eroded)))
snap_dilated = snap_eroded.filter(ImageFilter.MaxFilter(3))
snap_dilated.show()
print("After dilation, got median, mean as {:.2f}, {:.2f}.".format(np.median(snap_dilated), np.mean(snap_dilated)))


After erosion, got median, mean as 0.00, 0.04.
After dilation, got median, mean as 0.00, 0.12.


## References:
1. A comprehensive DIY [guide](http://drsol.com/~deid/pi/camera/index.html) to Pi camera including many lesser-known techniques for image and video recording, processing and sharing.
2. A github [repo](https://gist.github.com/FutureSharks/ab4c22b719cdd894e3b7ffe1f5b8fd91) for pro motion detection with OpenCV.
3. A stackoverflow.com [post](https://stackoverflow.com/questions/31064974/whats-the-fastest-way-to-threshold-a-numpy-array) upon thresholding with operations upon numpy arrays.